In [ ]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torchvision.transforms.functional import to_tensor
from torch.nn import CTCLoss
import string
from tqdm import tqdm
import editdistance

In [97]:
BATCH_SIZE = 16
IMG_HEIGHT = 32
IMG_WIDTH = 128
EPOCHS = 50
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CHARS = string.ascii_uppercase + string.digits+string.ascii_lowercase+"-_' "

In [ ]:
class LabelConverter:
    def __init__(self, characters):
        """
        characters: string of all characters in the dataset (excluding CTC blank)
        """
        self.characters = characters
        self.char2idx = {char: i + 1 for i, char in enumerate(characters)}  # 0 is reserved for blank
        self.idx2char = {i + 1: char for i, char in enumerate(characters)}
        self.blank = 0  # CTC requires blank token at index 0
        self.idx2char[self.blank] = ''  # for decoding

    def encode(self, text):
        """
        Convert a text string to a tensor of indices.
        Returns: Tensor of shape [len(text)]
        """
        return torch.tensor([self.char2idx[char] for char in text], dtype=torch.long)

    def decode(self, preds, merge_repeated=True):
        """
        Decode model output probabilities/logits into strings.
        Args:
            preds: Tensor of shape [T, B, C] (logits before/after softmax)
            merge_repeated: whether to remove repeated characters

        Returns:
            List of decoded strings, length = B
        """
        # Take argmax over class dimension
        pred_indices = preds.argmax(2).permute(1, 0)  # [B, T]
        results = []

        for indices in pred_indices:
            decoded = []
            prev_idx = None
            for idx in indices:
                idx = idx.item()
                if idx == self.blank:
                    prev_idx = None  # reset on blank
                    continue
                if merge_repeated and idx == prev_idx:
                    continue
                decoded.append(self.idx2char.get(idx, ''))
                prev_idx = idx
            results.append(''.join(decoded))
        return results
converter=LabelConverter(CHARS)

In [ ]:
class CRNNDataset(Dataset):
    def __init__(self, csv_path, transform=None):
        df = pd.read_csv(csv_path)
        self.paths = df['FILENAME'].values
        self.labels = df['IDENTITY'].values
        self.transform = transform or transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        img = self.transform(img)
        label = converter.encode(self.labels[idx])
        return {'image': img, 'label': label}

In [108]:
def collate_fn(batch):
    images = [item['image'] for item in batch]
    labels = [item['label'] for item in batch]
    label_lengths = torch.tensor([len(label) for label in labels], dtype=torch.long)
    images = torch.stack(images)
    labels_concat = torch.cat(labels)
    return images, labels_concat, label_lengths

In [109]:
class CRNN(nn.Module):
    def __init__(self, num_classes):
        super(CRNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(512, 512, 2, 1, 0), nn.ReLU(),
        )
        self.rnn1 = nn.LSTM(512, 256, bidirectional=True, batch_first=True)
        self.rnn2 = nn.LSTM(512, 256, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(512, num_classes + 1)  # +1 for CTC blank

    def forward(self, x):
        x = self.cnn(x)  # [B, 512, 1, W]
        x = x.squeeze(2).permute(0, 2, 1)  # [B, W, 512]
        x, _ = self.rnn1(x)  # [B, W, 512]
        x, _ = self.rnn2(x)  # [B, W, 512]
        x = self.fc(x)  # [B, W, C]
        return x.permute(1, 0, 2)  # [W, B, C] for CTC

In [110]:
def calculate_cer(pred, target):
    """Character Error Rate = (Edit Distance) / (Length of GT)"""
    if len(target) == 0:
        return 1.0 if len(pred) > 0 else 0.0
    return editdistance.eval(pred, target) / len(target)

def calculate_word_accuracy(preds, targets):
    """Word Accuracy = exact match rate"""
    correct = sum([p == t for p, t in zip(preds, targets)])
    return correct / len(targets) if targets else 0.0

In [114]:
def validate(model, val_loader, criterion, converter, print_samples=False):
    model.eval()
    val_loss = 0
    total_cer = 0
    total_words = 0
    correct_words = 0

    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, labels, label_lengths in val_loader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            preds = model(images)
            input_lengths = torch.full(size=(images.size(0),), fill_value=preds.size(0), dtype=torch.long)

            loss = criterion(preds, labels, input_lengths, label_lengths)
            val_loss += loss.item()

            # Decode predictions and labels
            decoded_preds = converter.decode(preds,merge_repeated=True)
            split_labels = []
            i = 0
            for length in label_lengths:
                text = ''.join([converter.idx2char[int(c.item())] for c in labels[i:i+length]])
                split_labels.append(text)
                i += length

            all_preds.extend(decoded_preds)
            all_targets.extend(split_labels)

            # Print sample predictions
            if print_samples:
                for pred, target in zip(decoded_preds[:3], split_labels[:3]):
                    print(f"GT: {target} | Pred: {pred}")

    # Compute metrics
    total_cer = sum(calculate_cer(p, t) for p, t in zip(all_preds, all_targets)) / len(all_targets)
    word_acc = calculate_word_accuracy(all_preds, all_targets)
    avg_loss = val_loss / len(val_loader)

    return avg_loss, total_cer, word_acc


In [ ]:
def train():
    # Datasets
    train_dataset = CRNNDataset(r'C:\Users\Raihan\OneDrive\Desktop\DPIIT HACKATHON\cvsi_10k_subset.csv')
    val_dataset = CRNNDataset(r'C:\Users\Raihan\OneDrive\Desktop\DPIIT HACKATHON\cvsi_val.csv')

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

    model = CRNN(num_classes=len(CHARS)).to(DEVICE)
    criterion = CTCLoss(blank=0, zero_infinity=True)
    optimizer = optim.AdamW(model arameters(), lr=1e-3)

    best_val_loss = float('inf')

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0

        for images, labels, label_lengths in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            preds = model(images)
            input_lengths = torch.full(size=(images.size(0),), fill_value=preds.size(0), dtype=torch.long)

            loss = criterion(preds, labels, input_lengths, label_lengths)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # ---- VALIDATION ----
        val_loss, val_cer,val_acc = validate(model, val_loader, criterion, converter)

        print(f"[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}|Val CER: {val_cer:.2f}")

        

    print("Training complete.")
    return model


In [116]:
model=train()

Epoch 1: 100%|██████████| 625/625 [01:16<00:00,  8.21it/s]


[Epoch 1] Train Loss: 3.2414 | Val Loss: 3.2285 | Val Acc: 0.00|Val CER: 1.00


Epoch 2: 100%|██████████| 625/625 [01:22<00:00,  7.60it/s]


[Epoch 2] Train Loss: 3.1557 | Val Loss: 3.3380 | Val Acc: 0.00|Val CER: 1.00


Epoch 3: 100%|██████████| 625/625 [01:19<00:00,  7.90it/s]


...

[Epoch 48] Train Loss: 0.0466 | Val Loss: 0.8017 | Val Acc: 0.58|Val CER: 0.12


Epoch 49: 100%|██████████| 625/625 [00:42<00:00, 14.63it/s]


[Epoch 49] Train Loss: 0.0440 | Val Loss: 0.6097 | Val Acc: 0.59|Val CER: 0.12


Epoch 50: 100%|██████████| 625/625 [00:42<00:00, 14.72it/s]


[Epoch 50] Train Loss: 0.0487 | Val Loss: 0.7357 | Val Acc: 0.58|Val CER: 0.12
Training complete.


In [ ]:
optimizer=optim.AdamW
torch.save({
    'epoch': 50,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, 'crnn_ctc_checkpoint.pth')

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3153723720.py, line 2)